In [46]:
import numpy as np
import csv

In [47]:
#sigmoid function defination
def sigmoid(z):
    s=1.0/(1+np.exp(-z))
    return s

#softmax function for multiclass classification
def softmax(z):
    s=np.exp(z)/np.sum(np.exp(z),axis=0)
    return s

###### Accuracy Function

In [48]:
#accuracy funciton
def accuracy(y,pred):
    count=0.0
    for i in range(0,y.shape[1]):
        if(y[0][i]==pred[i]):
            count=count+1
    print (count)
    return count*100/y.shape[1]

In [49]:
#initializing the layer sizes, assuming the hidden layer has 4 units
def layer_sizes(X,Y,n_h):
    n_x	= X.shape[0]
    n_y = Y.shape[0]
    n_h = n_h
    return (n_x,n_h,n_y)

In [50]:
#initializing weights and biases for layer 1 and layer 2
def initialize_paramteres(n_x,n_h,n_y):
    W1=np.random.randn(n_h,n_x)*0.01
    b1=np.zeros((n_h,1))
    W2=np.random.randn(n_y,n_h)*0.01
    b2=np.zeros((n_y,1))

    assert(W1.shape==(n_h,n_x))
    assert(b1.shape==(n_h,1))
    assert(W2.shape==(n_y,n_h))
    assert(b2.shape==(n_y,1))

    parameters={"W1":W1,
                "b1":b1,
                "W2":W2,
                "b2":b2}
    return parameters

In [51]:
#forward propagation to compute activation values. 13 is the number of classes 
def forward_propagation	(X,parameters):
    W1=parameters["W1"]
    b1=parameters["b1"]
    W2=parameters["W2"]
    b2=parameters["b2"]
    
    Z1=np.dot(W1,X)+b1
    A1=np.tanh(Z1)
    Z2=np.dot(W2,A1)+b2
    A2=softmax(Z2)


    assert(A2.shape==(13,X.shape[1]))

    cache={"Z1":Z1,
            "A1":A1,
            "Z2":Z2,
            "A2":A2}
    return A2,cache

In [52]:
#computes cost based on softmax function
def compute_cost(A2,Y,parameters):
    m=Y.shape[1]
    logprobs=(np.multiply(np.log(A2),Y))
    cost=-np.sum(logprobs)/m
    cost=float(cost)
    assert(isinstance(cost,float))

    return cost

In [53]:
#computes gradients for learning
def backward_propagation(parameters,cache,X,Y):
    m=X.shape[1]
    W1=parameters["W1"]
    W2=parameters["W2"]

    A1=cache["A1"]
    A2=cache["A2"]

    dZ2=A2-Y
    dW2=np.dot(dZ2,A1.T)/m
    db2=np.sum(dZ2,axis=1,keepdims=True)/m
    dZ1=np.dot(W2.T,dZ2)*(1-np.power(A1,2))
    dW1=np.dot(dZ1,X.T)/m
    db1=np.sum(dZ1,axis=1,keepdims=True)/m

    grads={"dW1":dW1,
           "db1":db1,
           "dW2":dW2,
           "db2":db2}
    return grads

In [54]:
#update parameters based on learning rate and gradients dW and db
def update_paramters(parameters,grads,learning_rate):
    W1=parameters["W1"]
    b1=parameters["b1"]
    W2=parameters["W2"]
    b2=parameters["b2"]

    dW1=grads["dW1"]
    db1=grads["db1"]
    dW2=grads["dW2"]
    db2=grads["db2"]
    
    W1=W1-learning_rate*dW1
    b1=b1-learning_rate*db1
    W2=W2-learning_rate*dW2
    b2=b2-learning_rate*db2

    parameters={"W1":W1,
                "b1":b1,
                "W2":W2,
                "b2":b2}
    return parameters

### Model for NN

In [55]:
def nn_model(X,Y,n_h,num_iterations=1000, print_cost=False):
    n_x,n_h,n_y=layer_sizes(X,Y,n_h)
    print("size of imput layer is n_x ="+str(n_x))
    print("size of hidden layer is n_xh ="+str(n_h))
    print("size of output layer is n_y ="+str(n_y))

    parameters= initialize_paramteres(n_x,n_h,n_y)


    for i in range(0,num_iterations):

        A2,cache=forward_propagation(X,parameters)
        cost =compute_cost(A2,Y,parameters)
        grads=backward_propagation(parameters,cache,X,Y)
        parameters=update_paramters(parameters,grads,0.001)
        if print_cost and i%100==0:
            print("Cost after iteration %i:%f" %(i,cost))
            #print(A2.shape)
            #print(A2)
            #print(np.sum(A2,axis=0).shape)
            #print(np.sum(A2, axis=0))



    return parameters

##### Predict Function

In [56]:
def predict(parameters, X):
    A2, cache = forward_propagation(X,parameters)
    print(np.transpose(A2))
    predictions=np.zeros(A2.shape[1])
    predictions=np.argmax(A2,axis=0)+1
    return predictions

#####  Loading the dataset

In [57]:
reader=csv.reader(open("reduced_features.csv","r"),delimiter=",")
X=list(reader)
X=np.array(X)
X=X.astype(np.float)

In [58]:
reader=csv.reader(open("target_output.csv","r"),delimiter=",")
Y=list(reader)
Y=np.array(Y)
Y=Y.astype(np.int)

###### Feature Scaling

In [59]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X=sc.fit_transform(X)

In [60]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(X,Y,test_size=0.1, random_state=0)


Y_train=np.transpose(Y_train)

In [61]:
#one hot encoding for multiclass
one_hot_encoded=list()
for value in range (0,Y_train.shape[1]):
    out=list()
    out=[0 for i in range(13)]
    out[Y_train[0][value]-1]=1
    one_hot_encoded.append(out)

In [62]:
Y_train=one_hot_encoded
Y_train=np.array(Y_train)


X_train=np.transpose(X_train)
Y_train=np.transpose(Y_train)

X_test=np.transpose(X_test)
Y_test=np.transpose(Y_test)


print("shape of x ="+str(X_train.shape))
print("shape of y ="+str(Y_train.shape))

shape of x =(175, 406)
shape of y =(13, 406)


In [63]:
#running the model for given number of iterations
parameters = nn_model(X_train, Y_train, 70, num_iterations=10000, print_cost=True)


size of imput layer is n_x =175
size of hidden layer is n_xh =70
size of output layer is n_y =13
Cost after iteration 0:2.566388
Cost after iteration 100:2.537709
Cost after iteration 200:2.509446
Cost after iteration 300:2.481525
Cost after iteration 400:2.453867
Cost after iteration 500:2.426394
Cost after iteration 600:2.399020
Cost after iteration 700:2.371652
Cost after iteration 800:2.344188
Cost after iteration 900:2.316515
Cost after iteration 1000:2.288512
Cost after iteration 1100:2.260046
Cost after iteration 1200:2.230979
Cost after iteration 1300:2.201174
Cost after iteration 1400:2.170498
Cost after iteration 1500:2.138837
Cost after iteration 1600:2.106106
Cost after iteration 1700:2.072267
Cost after iteration 1800:2.037339
Cost after iteration 1900:2.001416
Cost after iteration 2000:1.964667
Cost after iteration 2100:1.927336
Cost after iteration 2200:1.889732
Cost after iteration 2300:1.852201
Cost after iteration 2400:1.815100
Cost after iteration 2500:1.778760
Cost 

In [64]:
#using the paramters values to predict future values i.e values of testing set
predictions=predict(parameters,X_test)

#predictions for testing set
print("predictions = "+str(predictions))


[[8.09410581e-01 1.38080382e-02 1.01261536e-02 1.27751858e-02
  2.49668517e-02 2.80416420e-02 1.37552656e-02 1.29094373e-02
  5.79233430e-03 2.36154303e-02 1.22961195e-02 1.18280282e-02
  2.06749327e-02]
 [2.84086777e-01 2.81759361e-01 2.34719148e-02 5.93372469e-02
  2.63517948e-02 5.74678167e-02 3.19628469e-02 3.53324266e-02
  4.72443297e-02 2.67808138e-02 3.25057730e-02 4.50652120e-02
  4.86336865e-02]
 [6.86383484e-03 1.92983441e-03 1.37627516e-02 4.04929499e-03
  1.37760340e-02 1.56843847e-02 1.00604822e-02 9.89353234e-03
  8.30249716e-03 8.77812285e-01 1.34410684e-02 8.51602640e-03
  1.59079740e-02]
 [9.04091181e-01 3.55544252e-02 2.13744248e-03 6.20637619e-03
  3.45654647e-03 1.62599286e-02 4.77424923e-03 4.68977962e-03
  5.52855334e-03 6.42372710e-04 3.94729414e-03 5.19397631e-03
  7.51787485e-03]
 [4.91854646e-01 1.45293718e-01 2.08808327e-02 8.24896971e-02
  4.10557547e-02 3.62912597e-02 2.63413705e-02 2.77111800e-02
  2.24661570e-02 1.10521276e-02 2.48225377e-02 3.45505562e-0

In [65]:
#Accuracy of classification for testing set
print("Accuracy = "+str(accuracy(Y_test,predictions))+ " %")

27.0
Accuracy = 58.69565217391305 %


In [67]:
#Database:  Arrhythmia

#   Class code :   Class   :                       Number of instances:
#   01             Normal                          245
#   02             Ischemic changes (Coronary Artery Disease)   44
#   03             Old Anterior Myocardial Infarction           15
#   04             Old Inferior Myocardial Infarction           15
#   05             Sinus tachycardy                     13
#   06             Sinus bradycardy                     25
#   07             Ventricular Premature Contraction (PVC)       3
#   08             Supraventricular Premature Contraction        2
#   09             Left bundle branch block                  9   
#   10             Right bundle branch block                50
#   11             1. degree AtrioVentricular block              0   
#   12             2. degree AV block                    0
#   13             3. degree AV block                    0
#   14             Left ventricule hypertrophy               4
#   15             Atrial Fibrillation or Flutter                5
#   16             Others                           22